In [4]:
# 옵션기능추가; 옵션값에 맞춰서 이미지통합

import os
import tkinter.ttk as ttk
import tkinter.messagebox as msgbox
from tkinter import * #all부분에 정의하지 않으면 (서브모듈은) 별도의 import 필요
from tkinter import filedialog  #파일다이얼로그 가져다 쓰기
from PIL import Image


root = Tk()
root.title("나도 코딩! 사진 이어붙이기")

##파일추가 파일다이얼로그는 갖다쓰면됨
def add_file():
    files = filedialog.askopenfilenames(title="여러개의 파일을 선택하세요",\
        filetypes=(("PNG 파일","*.png"),("모든파일","*.*")),\
        initialdir=r"C:\workspace\cakd3\programming\hj_individual\Nado")  #r-Rowstring
    #askopenfilenames,여러개의 파일 선택할 수 있도록/initialdir 파일추가버튼누르면 뜨는 최초의 경로 - 지정
    #사용자가 선택한 파일 목록
    for file in files:
        list_file.insert(END, file)  #리스트프레임에 들어가도록

###파일 선택 삭제
def del_file():
    # 파일 선택했을때 그 파일의 인덱스 reversed;뒤에서부터 가져오기
    for index in reversed(list_file.curselection()):
        list_file.delete(index)

##저장 경로 (폴더) 기능
def browse_dest_path():
    folder_selected = filedialog.askdirectory()
    if folder_selected == '':  #사용자가 취소를 누룰 때, 취소 : ''(빈문자열)
        return
    #print(folder_selected) -> 폴더경로 확인, 눌렀을때랑 취소눌렀을때 출력으로 확인
    e_dest_path.delete(0,END)
    e_dest_path.insert(0, folder_selected)

###이미지 통합
def merge_image():
    try:
        #가로넓이
        img_width = cmb_width.get()
        if img_width == "원본유지":
            img_width = -1 # -1일때는 원본 기준으로
        else:
            img_width = int(img_width) #정수형으로

        #간격
        img_space = cmb_space.get()
        if img_space == "좁게":
            img_space = 30
        elif img_space == "보통":
            img_space = 60
        elif img_space == "넓게":
            img_space = 90
        else: 
            img_space = 0

        #포맷
        img_format = cmb_format.get().lower() #PNG, JPG, BMP 값을 받아워서 소문자로 변경

        ######################################
        images = [Image.open(x) for x in list_file.get(0,END)]
        #가로넓이 업데이트, 이미지 사이즈를 리스트에 넣어서 하나씩 처리
        image_sizes = [] # 이미지들의 값을 image_sizes = {(width1, height1),(width2, height2)...} 튜플로 처리 
        if img_width > -1 :
            #width 값 변경
            image_sizes = [int(img_width), int(img_width * x.size[1]/x.size[0])]
        else:
            #원본 사이즈 사용
            image_sizes = [(x.size[0], x.size[1]) for x in images]

        #### width 값 변경 계산식
        # 100*60 이미지 -> width의 80으로 줄이면 높이는?
        # 원본 width : 원본 height = 변경 width : 변경 height
        
        #코드에 대입
        # size[0] : size[1] = img_width : img_width*size[1]/size[0]


        # size -> size[0] : width/ size[1] : height 
        # 이미지 객체의 사이즈 번호 여러개의 이미지값 중 가장 넓은 width를 처리하고 모든 이미지의 전체 height 를 알기위해 값 받아와야함
        widths, heights = zip(*(image_sizes))

        #사진을 다 넣는 긴 스케치북 준비하기 위해 최대 가로 길이와 높이들의 합을 구함.
        max_width, total_height = max(widths), sum(heights)


        #빈 스케치북 만들기
        if img_space > 0: #이미지 간격 옵션 적용
            total_height += (img_space * (len(images)-1))
        result_img = Image.new("RGB",(max_width, total_height),(255,255,255)) #RGB 배경 흰색 255
        #x는 그대론데 y를 기준으로 하나씩 추가하면서 이미지 늘려가기위해 쓰는 변수
        y_offset = 0
        for idx, img in enumerate(images):
            # width가 원본유지가 아닐 경우 사이즈 적용
            if img_width > -1:
                img = img.resize(image_sizes[idx])

            result_img.paste(img, (0,y_offset))
            y_offset += (img.size[1] + img_space) #height 값 + 사용자가 지정한 간격

            progress = (idx + 1)/len(images) * 100 
            p_var.set(progress)
            progress_bar.update()  #ui 업데이트

            
        # 실제 실행, 실제로 이미지가 저장되는 경로  
        # 포맷 옵션 적용  
        file_name = "nado_photo." + img_format
        dest_path = os.path.join(e_dest_path.get(), file_name)
        result_img.save(dest_path)
        msgbox.showinfo("알림","작업이 완료되었습니다.")
    except Exception as e:  #C드라이브나 Z드라이브에는 추가할 수 없기때문에 예외처리
        msgbox.showerror("에러", e)


##시작버튼 기능
def start():
    #각 옵션들 값을 확인
    # print("가로넓이: ", cmb_width.get())
    # print("간격: ", cmb_space.get())
    # print("포맷 :", cmb_format.get())

    #파일 목록 확인, 파일이 하나도 없을때
    if list_file.size() == 0:
        msgbox.showwarning("경고","이미지 파일을 추가하세요.")
        return

    #저장 경로 확인,저장경로가 비어있을 경우
    if len(e_dest_path.get()) == 0:
        msgbox.showwarning("경고","저장 경로를 선택하세요.")
        return

    #이미지 통합 작업
    merge_image()



#파일 프레임, 맨 위의 (파일 추가, 선택 삭체 )
file_frame = Frame(root)
file_frame.pack(fill='x', pady=5, padx=5) #padx,y 간격띄우기

btn_add_file = Button(file_frame, padx = 5, pady = 5, width = 12, text="파일 추가",command=add_file)
btn_add_file.pack(side="left")
btn_del_file = Button(file_frame, padx = 5, pady = 5, width = 12, text="파일 삭제",command=del_file)
btn_del_file.pack(side="right")


#리스트 프레임
list_frame = Frame(root)
list_frame.pack(fill="both", pady=5, padx=5)

#리스트 스크롤바
scr= Scrollbar(list_frame)
scr.pack(side="right", fill="y")

list_file = Listbox(list_frame, selectmode="extended", height=15, yscrollcommand=scr.set)
list_file.pack(side="left", fill="both", expand=True)
scr.config(command=list_file.yview)


#저장 경로 프레임
path_frame = LabelFrame(root, text="저장 경로")
path_frame.pack(fill='x', pady=5, padx=5, ipady=5)

e_dest_path = Entry(path_frame)
e_dest_path.pack(side="left", fill="x", expand=True, pady=5, padx=5, ipady=4) 

btn_dest_path = Button(path_frame, text = "찾아보기", width =10, command=browse_dest_path)
btn_dest_path.pack(side="right", pady=5, padx=5)


#옵션 프레임
frame_option = LabelFrame(root, text="옵션")
frame_option.pack(pady=5, padx=5, ipady=5)

#1. 가로 넓이 옵션
# 가로 넓이 레이블
lbl_width = Label(frame_option,text="가로넓이",width=8)
lbl_width.pack(side="left", pady=5, padx=5) 

# 가로 넓이 콤포
opt_width = ["원본유지", "1024", "800", "640"]
cmb_width = ttk.Combobox(frame_option, state="readonly", values=opt_width, width=10)
cmb_width.current(0)
cmb_width.pack(side="left", pady=5, padx=5)


#2. 간격 옵션
#간격 옵션 레이블
lbl_space = Label(frame_option,text="간격",width=8)
lbl_space.pack(side="left", pady=5, padx=5)

#간격 옵션 콤보
opt_space = ["없음", "좁게", "보통", "넓게"]
cmb_space = ttk.Combobox(frame_option, state="readonly", values=opt_space, width=10)
cmb_space.current(0)
cmb_space.pack(side="left", pady=5, padx=5)

#3. 파일 포맷 옵션
#파일 포맷 레이블
lbl_format = Label(frame_option,text="포맷",width=8)
lbl_format.pack(side="left", pady=5, padx=5)

#파일 포맷 콤보
opt_format = ["PNG", "JPG", "BMP"]
cmb_format = ttk.Combobox(frame_option, state="readonly", values=opt_format, width=10)
cmb_format.current(0)
cmb_format.pack(side="left", pady=5, padx=5)


#진행상황 프로그레스바
frame_progress = LabelFrame(root, text="진행상황")
frame_progress.pack(fill="x", pady=5, padx=5, ipady=5)

p_var = DoubleVar()
progress_bar = ttk.Progressbar(frame_progress,maximum=100,variable=p_var)
progress_bar.pack(fill="x", pady=5, padx=5)



#실행 프레임
frame_run = Frame(root)
frame_run.pack(fill="x", pady=5, padx=5)

btn_close = Button(frame_run,padx=5,pady=5, text="닫기",width=12,command=root.quit)
btn_close.pack(side="right", pady=5, padx=5)
btn_start = Button(frame_run,padx=5,pady=5, text="시작",width=12,command=start)
btn_start.pack(side="right", pady=5, padx=5)


root.resizable(False,False) #너비, 높이 창 크기 변경 불가
root.mainloop()
